In [1]:
import pandas as pd
import numpy as np
import os
import re
from typing import Dict, List, Tuple


In [2]:
# 201 => 1,2,3,4,9 호선
line_1 = [f"0{num}" for num in range(150, 160)]
line_2 = [f"0{num}" for num in range(201, 251)]
line_3 = [f"0{num}" for num in range(309, 343)]

line_4 = [f"0{num}" for num in range(409, 435)]
line_4.append("0405")
line_4.append("0406")
line_4.append("0408")
line_4 = sorted(set(line_4))


# 9호선 일부 205번 코드
line_9 = [f"0{num}" for num in range(4126, 4139)]   # 201 교통수단 코드
for i in range(4101, 4126):                         # 205
    line_9.append(f"0{i}")
line_9 = sorted(set(line_9))

names_line_1: List[str] = [
    "서울역", "시청", "종각", "종로3가", "종로5가", "동대문", "신설동", "제기동", "청량리(지하)", "동묘앞",
] 
names_line_2: List[str] = [
    "시청", "을지로입구", "을지로3가", "을지로4가", "동대문역사문화공원", "신당", "상왕십리", "왕십리(성동구청)",
    "한양대", "뚝섬", "성수", "건대입구", "구의", "강변", "잠실나루", "잠실", "잠실새내", "종합운동장",
    "삼성", "선릉", "역삼", "강남", "교대", "서초", "방배", "사당", "낙성대", "서울대입구", "봉천",
    "신림", "신대방", "구로디지털단지", "대림", "신도림", "문래", "영등포구청", "당산", "합정", "홍대입구",
    "신촌", "이대", "아현", "충정로", "용답", "신답", "신설동", "도림천", "양천구청", "신정네거리", "용두"
]
names_line_3: List[str] = [
    "지축", "구파발", "연신내", "불광", "녹번", "홍제", "무악재", "독립문", "경복궁", "안국", "종로3가",
    "을지로3가", "충무로", "동대입구", "약수", "금호", "옥수", "압구정", "신사", "잠원", "고속터미널",
    "교대", "남부터미널", "양재", "매봉", "도곡", "대치", "학여울", "대청", "일원", "수서", "가락시장",
    "경찰병원", "오금"
]
names_line_4: List[str] = [
    "오남", "진접", "별내별가람", "당고개", "상계", "노원", "창동", "쌍문", "수유(강북구청)", "미아",
    "미아사거리", "길음", "성신여대입구", "한성대입구", "혜화", "동대문", "동대문역사문화공원", "충무로",
    "명동", "회현", "서울역", "숙대입구", "삼각지", "신용산", "이촌", "동작", "이수", "사당", "남태령"
]
names_line_9: List[str] = [
    "개화", "김포공항", "공항시장", "신방화", "마곡나루", "양천향교", "가양", "증미", "등촌", "염창",
    "신목동", "선유도", "당산", "국회의사당", "여의도", "샛강", "노량진", "노들", "흑석(중앙대입구)",
    "동작", "구반포", "신반포", "고속터미널", "사평", "신논현", "언주", "선정릉", "삼성중앙", "봉은사",
    "종합운동장", "삼전", "석촌고분", "석촌", "송파나루", "한성백제", "올림픽공원(한국체대)", "둔촌오륜",
    "중앙보훈병원"
]

# --- 3) 유틸: 매핑 생성기 + 검증 ---
def build_map(codes: List[str], names: List[str]) -> Dict[str, str]:
    if len(codes) != len(names):
        raise ValueError(f"코드({len(codes)}) 수와 역명({len(names)}) 수가 다릅니다.")
    # 중복 코드 체크
    dup = [c for c in codes if codes.count(c) > 1]
    if dup:
        raise ValueError(f"중복 코드 존재: {sorted(set(dup))}")
    # 중복 역명은 허용할 수도 있지만(환승역 등 동일 표기), 필요하면 아래처럼 경고/에러 처리
    # dup_names = [n for n in names if names.count(n) > 1]
    # if dup_names: print("경고: 중복 역명:", sorted(set(dup_names)))
    return dict(zip(codes, names))

In [3]:
# --- 4) 라인별 딕셔너리 생성 ---
map_line_1 = build_map(line_1, names_line_1)
map_line_2 = build_map(line_2, names_line_2)
map_line_3 = build_map(line_3, names_line_3)
map_line_4 = build_map(line_4, names_line_4)
map_line_9 = build_map(line_9, names_line_9)

In [4]:
map_line_1

{'0150': '서울역',
 '0151': '시청',
 '0152': '종각',
 '0153': '종로3가',
 '0154': '종로5가',
 '0155': '동대문',
 '0156': '신설동',
 '0157': '제기동',
 '0158': '청량리(지하)',
 '0159': '동묘앞'}

In [5]:
# --- 5) 통합 딕셔너리 (코드 → 역명) ---
def merge_maps(*maps: Dict[str, str]) -> Dict[str, str]:
    merged: Dict[str, str] = {}
    for m in maps:
        for k, v in m.items():
            if k in merged and merged[k] != v:
                # 서로 다른 라인에서 같은 코드가 쓰인다면 충돌. 정책 결정 필요.
                raise ValueError(f"코드 충돌: {k} -> '{merged[k]}' vs '{v}'")
            merged[k] = v
    return merged

code_to_station = merge_maps(map_line_1, map_line_2, map_line_3, map_line_4, map_line_9)

def station_name(code: str) -> str:
    return code_to_station.get(code, "<UNKNOWN>")

def codes_by_station(name: str) -> List[str]:
    # 동일 역명이 여러 코드에 매핑될 수 있음(환승역 다중 코드 등)
    return [c for c, n in code_to_station.items() if n == name]

In [6]:
may_path = os.path.join(os.getcwd(),  "train_pars_final")
data_path_list = [os.path.join(may_path, i)for i in os.listdir(may_path)]
one_file_len = 100_000
print(len(data_path_list) * one_file_len)

0


In [7]:
tmp = data_path_list[0]
print(tmp)
data = np.load(tmp, allow_pickle=True)
print(data.shape)
print(data[0])

IndexError: list index out of range

In [ ]:
df = pd.DataFrame(data, columns=['user_code', 'Transaction_code', 'InTime','InInformation','OutTime','OutInformation','TransactionID', 'TransferCount'])
df_2ndline = df[(df['Transaction_code'].apply(lambda x: x[0] == 201))]
df_2ndline_sorted = df_2ndline.sort_values(by=['user_code', 'InTime'], ascending=True)
print(df_2ndline.shape)

(35539, 8)


In [ ]:
df_2ndline_sorted = df_2ndline[df_2ndline['InInformation'].apply(lambda x: x[1] == 202)]
print(df_2ndline_sorted)

                                          user_code Transaction_code  \
588    KDiZ/m93a16ztIi0+LJ239MNyMTRFqgtpBWPTWVAkj4=    (201, 알 수 없음)   
1070   KyZMXbTzgIo2x0VE/ss1qElp2ySidbmgZCMlJ8jXGUk=    (201, 알 수 없음)   
1164   JQX1oIlyfv071Wz0iyL4zAZR3Wx0iZWAM4ox4OB6BEw=    (201, 알 수 없음)   
1399   KyZaWs6DugnEjuRMrcIHlcQU5rwykFy9APJGBs9xSxU=    (201, 알 수 없음)   
1766   LlUwXha7mliZcKGA1VS+YAoc8x3CfJEQ8EAdEiWGr+Q=    (201, 알 수 없음)   
...                                             ...              ...   
99605  Iz7T68Tkb3Q/QwM4XJg7sCtxRrowIxD1Uy0tjmkDhGw=    (201, 알 수 없음)   
99630  ICJHuRsb3O02UiGmR3sOIzFGnr7+rPF3MropNpYHSgM=    (201, 알 수 없음)   
99631  ICJHuRsb3O02UiGmR3sOIzFGnr7+rPF3MropNpYHSgM=    (201, 알 수 없음)   
99654  IEdIAdvocdlIlWTeoYcPJvrBrOz+/aPuGe5ZQiFZ5+Q=    (201, 알 수 없음)   
99910  HTFApO7+M5zIiVGoDJzrY+97zwQa53kLkzuXvsvmops=    (201, 알 수 없음)   

                   InTime      InInformation             OutTime  \
588   2022-05-21 06:43:39  (nan, 202, 을지로입구) 2022-05-21 07:44:01   

In [ ]:
print(df_2ndline_sorted[:4])
print()
print(len(set(df_2ndline_sorted['user_code'])))

                                         user_code Transaction_code  \
588   KDiZ/m93a16ztIi0+LJ239MNyMTRFqgtpBWPTWVAkj4=    (201, 알 수 없음)   
1070  KyZMXbTzgIo2x0VE/ss1qElp2ySidbmgZCMlJ8jXGUk=    (201, 알 수 없음)   
1164  JQX1oIlyfv071Wz0iyL4zAZR3Wx0iZWAM4ox4OB6BEw=    (201, 알 수 없음)   
1399  KyZaWs6DugnEjuRMrcIHlcQU5rwykFy9APJGBs9xSxU=    (201, 알 수 없음)   

                  InTime      InInformation             OutTime  \
588  2022-05-21 06:43:39  (nan, 202, 을지로입구) 2022-05-21 07:44:01   
1070 2022-05-21 20:36:26  (nan, 202, 을지로입구) 2022-05-21 21:07:05   
1164 2022-05-21 14:50:10  (nan, 202, 을지로입구) 2022-05-21 15:06:14   
1399 2022-05-21 21:59:51  (nan, 202, 을지로입구) 2022-05-21 22:49:31   

         OutInformation TransactionID TransferCount  
588   (nan, 1208.0, 덕소)             1             1  
1070   (nan, 415.0, 미아)             2             0  
1164  (nan, 317.0, 경복궁)             1             0  
1399  (nan, 1706.0, 안양)             2             0  

377


In [ ]:
import glob
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from collections.abc import Iterable


# ========= 환경/스키마 설정 =========
data_dir  = may_path    # 탐색 루트 폴더
day_start_str = "20220501"
date_str  = "TCD_20220501_train_final_part"       # 분석 일자 (YYYY-MM-DD)
pattern   = f"{data_dir}/{date_str}*.npy"  # 하위 폴더 포함 탐색

day_start = pd.Timestamp(day_start_str)
day_end = day_start + pd.Timedelta(days=1)

def z4(x):  # 4자리 zero pad
    return str(int(x)).zfill(4)

def row_match_codes(val, codes:set[str]) -> bool:
    """
    val: Transaction_code 한 행의 값 (예: '0201', 201, ('0201','XXXX') 등)
    codes: 허용 코드 집합(문자열 4자리)
    """
    # 시퀀스(튜플/리스트/ndarray 등)인 경우: 구성요소 중 하나라도 매칭되면 True
    if isinstance(val, Iterable) and not isinstance(val, (bytes, bytearray)):
        try:
            for x in val:
                if pd.isna(x):
                    continue
                if z4(x) in codes:
                    return True
            return False
        except Exception:
            return False

    # 그 외 타입은 매칭 실패로 간주
    return False

def iter_day_files():
    return sorted(glob.glob(pattern))

# ========= 공통: 파일->DataFrame (부분 컬럼만) =========
def load_part_df(npy_path, need_count=False):
    data = np.load(npy_path, allow_pickle=True)
    df = pd.DataFrame(data, columns=['user_code', 'Transaction_code', 'InTime','InInformation','OutTime','OutInformation','TransactionID', 'TransferCount'])

    # 필요한 컬럼만 추출
    # df_selected_line = df[(df['Transaction_code'].apply(lambda x: x[0] == 201))]

    # np.datetime64 -> pandas datetime64[ns]
    # (np.datetime64이면 to_datetime에서 손실 없이 처리됨)
    df["InTime"] = pd.to_datetime(df["InTime"], errors="coerce")
    df["OutTime"] = pd.to_datetime(df["OutTime"], errors="coerce")

    return df

# ========= 1) 당일 고유 사용자 수 (옵션: 특정 호선만) =========
def unique_users_in_day(line_id=None):
    uniq = set()
    for p in iter_day_files():
        df = load_part_df(p, need_count=False)
        if line_id is not None:
            df = df[df['Transaction_code'].apply(lambda x: x[0] == 201)]
            if line_id == 1:
                df = df[df['InInformation'].apply(lambda x: z4(x[1]) in line_1)]
            elif line_id == 2:
                df = df[df['InInformation'].apply(lambda x: z4(x[1]) in line_2)]
            elif line_id == 3:
                df = df[df['InInformation'].apply(lambda x: z4(x[1]) in line_3)]
            elif line_id == 4:
                df = df[df['InInformation'].apply(lambda x: z4(x[1]) in line_4)]
            else:
                df = df[df['InInformation'].apply(lambda x: z4(x[1]) in line_9)]
        if not df.empty:
            uniq.update(df["user_code"].unique().tolist())
    return len(uniq)

# ========= 2) 시간 버킷 집계 (이벤트 건수 or count 합) =========
def time_bucket_counts(freq="H", line_id=None):
    """
    freq: 'H' | '30T' | '10T'
    반환: DatetimeIndex (해당 일자 구간) 인덱스의 Series (int)
    """
    total = None
    for p in iter_day_files():
        df = load_part_df(p)
        if line_id is not None:
            df = df[df['Transaction_code'].apply(lambda x: x[0] == 201)]
            if line_id == 1:
                df = df[df['InInformation'].apply(lambda x: z4(x[1]) in line_1)]
            elif line_id == 2:
                df = df[df['InInformation'].apply(lambda x: z4(x[1]) in line_2)]
            elif line_id == 3:
                df = df[df['InInformation'].apply(lambda x: z4(x[1]) in line_3)]
            elif line_id == 4:
                df = df[df['InInformation'].apply(lambda x: z4(x[1]) in line_4)]
            else:
                df = df[df['InInformation'].apply(lambda x: z4(x[1]) in line_9)]
        if df.empty:
            continue

        df = df.set_index("InTime")
        part = df.resample(freq).size()              # 이벤트 건수

        total = part if total is None else total.add(part, fill_value=0)

    # 누락 구간 0 채우기 및 정렬
    if total is None:
        # 일자 전체 인덱스(빈 시리즈) 생성
        full_idx = pd.date_range(day_start, day_end, freq=freq, inclusive="left")
        return pd.Series(0, index=full_idx, dtype="int64")

    full_idx = pd.date_range(day_start, day_end, freq=freq, inclusive="left")
    total = total.reindex(full_idx, fill_value=0).sort_index().astype("int64")
    return total

# ================= 실행 예시 =================
# # (A) 당일 전체/특정 호선 고유 이용자 수
# all_users = unique_users_in_day(line_id=None)
# line2_users = unique_users_in_day(line_id=2)

# # (B) 1시간/30분/10분 버킷 집계 (이벤트 건수 기준)
# by_1h_all  = time_bucket_counts(freq="h",   line_id=None)
# by_30m_all = time_bucket_counts(freq="30min", line_id=None)
# by_10m_all = time_bucket_counts(freq="10min", line_id=None)

# # (C) 특정 호선(예: 2호선)
# by_1h_line2 = time_bucket_counts(freq="h", line_id=2)

# print("전체 고유 사용자 수:", all_users)
# print("2호선 고유 사용자 수:", line2_users)
# print(by_1h_all.head())
# print(by_1h_line2.head())


In [ ]:
import os, re, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from collections.abc import Iterable
from matplotlib import font_manager, rcParams

# ================= 기본 설정 =================
may_path = os.path.join(os.getcwd(), "202205", "train_pars_final_5")
root_path   = may_path                     # 전체 데이터 루트
recursive   = False                        # 하위폴더까지면 True로, 패턴에 ** 추가됨
out_dir     = os.path.join(root_path, "plots")
target_lines = [2, None]       # None=전체집계 포함(원하면 수정)
freqs = ("1min",) #("h", "30min", "10min", "1min")            # 저장할 시간 버킷

os.makedirs(out_dir, exist_ok=True)

# ---------------- (선택) 한글 폰트 ----------------
def setup_korean_font():
    candidates = ['Malgun Gothic','AppleGothic','NanumGothic','Noto Sans CJK KR','Noto Sans KR']
    avail = {f.name for f in font_manager.fontManager.ttflist}
    for c in candidates:
        if c in avail:
            rcParams['font.family'] = c
            break
    rcParams['axes.unicode_minus'] = False
setup_korean_font()

# ================ 라인 코드셋 ================
def z4(x):  # 4자리 zero-pad
    return str(int(x)).zfill(4)

# 필요 시 직접 정의되어 있는 line_1~line_9 리스트를 아래처럼 정규화해서 set[str]로 사용
# (이미 존재한다면 이 블록으로 교체/정규화 권장)
# line_1 = {z4(n) for n in range(150, 160)}
# line_2 = {z4(n) for n in range(201, 251)}
# line_3 = {z4(n) for n in range(309, 343)}
# line_4 = {z4(n) for n in range(409, 435)} | {"0405","0406","0408"}
# line_9 = {z4(n) for n in range(4101, 4139)}  # 4101~4138 (201/205 혼재 구간 포함)
LINE_CODESETS = {1: line_1, 2: line_2, 3: line_3, 4: line_4, 9: line_9}

# ================ 파일 탐색/날짜 추출 ================
def discover_dates(root_path:str, recursive:bool=False):
    pat = os.path.join(root_path, "**", "TCD_*_train_final_part*.npy") if recursive \
          else os.path.join(root_path, "TCD_*_train_final_part*.npy")
    files = glob.glob(pat, recursive=recursive)
    rx = re.compile(r"TCD_(\d{8})_train_final_part")
    dates = set()
    for p in files:
        m = rx.search(os.path.basename(p))
        if m: dates.add(m.group(1))
    return sorted(dates)

def iter_day_files(root_path:str, yyyymmdd:str, recursive:bool=False):
    prefix = f"TCD_{yyyymmdd}_train_final_part"
    pat = os.path.join(root_path, "**", f"{prefix}*.npy") if recursive \
          else os.path.join(root_path, f"{prefix}*.npy")
    return sorted(glob.glob(pat, recursive=recursive))

# ================ 안전 추출 함수 ================
def get_mode_code(x):
    """교통수단 코드(201/205 등) - InInformation의 첫 요소로 가정."""
    try:
        if isinstance(x, np.generic):
            x = x.item()
        if isinstance(x, Iterable) and not isinstance(x, (str, bytes, bytearray)):
            it = iter(x)
            return int(next(it))
        return int(x)
    except Exception: return None

def get_line_code_str4(x):
    """노선 코드(예: '0201') - InInformation의 둘째 요소로 가정."""
    try:
        if isinstance(x, np.generic): x = x.item()
        if isinstance(x, Iterable) and not isinstance(x, (str, bytes, bytearray)):
            it = iter(x)
            next(it)          # 첫 요소 skip
            second = next(it, None)
            return z4(second) if second is not None and not pd.isna(second) else None
        return z4(x)  # 비정상 스칼라 케이스 방어
    except Exception: return None

# ================ 로딩/필터/집계 ================
def load_part_df(npy_path:str, day_start:pd.Timestamp):
    data = np.load(npy_path, allow_pickle=True)   # object dtype이라 mmap 불가
    df = pd.DataFrame(
        data,
        columns=['user_code','Transaction_code','InTime','InInformation',
                 'OutTime','OutInformation','TransactionID','TransferCount']
    )
    df["InTime"] = pd.to_datetime(df["InTime"], errors="coerce")
    day_end = day_start + pd.Timedelta(days=1)
    df = df[(df["InTime"] >= day_start) & (df["InTime"] < day_end)]
    return df[["user_code",'Transaction_code',"InInformation","InTime"]]

def filter_by_line(df:pd.DataFrame, line_id:int|None):
    if line_id is None:
        return df
    allowed_modes = {201, 205} if line_id == 9 else {201}
    df = df[df["Transaction_code"].apply(lambda v: get_mode_code(v) in allowed_modes)]
    codes = LINE_CODESETS[line_id]
    df = df[df["InInformation"].apply(lambda v: (c := get_line_code_str4(v)) is not None and c in codes)]
    return df

def unique_users_in_day(root_path:str, yyyymmdd:str, line_id:int|None):
    day_start = pd.Timestamp(yyyymmdd)
    uniq = set()
    for p in iter_day_files(root_path, yyyymmdd, recursive):
        df = load_part_df(p, day_start)
        if df.empty: continue
        df = filter_by_line(df, line_id)
        if df.empty: continue
        uniq.update(df["user_code"].unique().tolist())
    return len(uniq)

def time_bucket_counts(root_path:str, yyyymmdd:str, freq:str, line_id:int|None):
    """freq: 'h' | '30min' | '10min'"""
    day_start = pd.Timestamp(yyyymmdd)
    total = None
    for p in iter_day_files(root_path, yyyymmdd, recursive):
        df = load_part_df(p, day_start)
        if df.empty: continue
        df = filter_by_line(df, line_id)
        if df.empty: continue
        s = (df.set_index("InTime").resample(freq).size().astype("int64"))
        total = s if total is None else total.add(s, fill_value=0)
    full_idx = pd.date_range(day_start, day_start + pd.Timedelta(days=1), freq=freq, inclusive="left")
    if total is None:
        return pd.Series(0, index=full_idx, dtype="int64")
    return total.reindex(full_idx, fill_value=0).sort_index().astype("int64")

# ================ 플로팅 ================
def save_time_series_plot(series:pd.Series, title:str, outpath:str,
                          xmin:pd.Timestamp, xmax:pd.Timestamp):
    fig, ax = plt.subplots(figsize=(9,5))
    ax.plot(series.index, series.values, marker='o')
    ax.set_title(title)
    ax.set_xlabel("Time")
    ax.set_ylabel("Usage Count")
    ax.set_xlim(xmin, xmax)
    locator = mdates.AutoDateLocator(minticks=8, maxticks=12)
    ax.xaxis.set_major_locator(locator)
    ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(locator))
    ax.grid(True)
    plt.tight_layout()
    fig.savefig(outpath, dpi=150)
    plt.close(fig)

def save_daily_plot(daily_series:pd.Series, title:str, outpath:str):
    fig, ax = plt.subplots(figsize=(9,5))
    ax.plot(daily_series.index, daily_series.values, marker='o')
    ax.set_title(title)
    ax.set_xlabel("Date")
    ax.set_ylabel("Unique Users")
    locator = mdates.AutoDateLocator(minticks=6, maxticks=10)
    ax.xaxis.set_major_locator(locator)
    ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(locator))
    ax.grid(True)
    plt.tight_layout()
    fig.savefig(outpath, dpi=150)
    plt.close(fig)

def line_label(line_id:int|None):
    return "all" if line_id is None else f"line{line_id}"

# ================ 메인 루프 ================
def process_all_days():
    dates = discover_dates(root_path, recursive)
    if not dates:
        raise RuntimeError("No dates found. Please check file name pattern/path.")

    for line_id in target_lines:
        # --- 1) 일자별 고유 사용자 수 집계/저장 ---
        # day_vals = []
        # for d in dates:
        #     cnt = unique_users_in_day(root_path, d, line_id)
        #     day_vals.append((pd.Timestamp(d), cnt))
        #     print(f"[{line_label(line_id)}] {d} unique_users = {cnt}")

        # daily_series = pd.Series(
        #     [v for _, v in day_vals],
        #     index=[t for t, _ in day_vals],
        #     name="unique_users"
        # ).sort_index()

        subdir = os.path.join(out_dir, line_label(line_id))
        os.makedirs(subdir, exist_ok=True)
        # # 그래프 + CSV 저장
        # save_daily_plot(daily_series,
        #                 f"Daily Unique Users ({line_label(line_id)})",
        #                 os.path.join(subdir, f"{line_label(line_id)}_daily_unique.png"))
        # daily_series.to_csv(os.path.join(subdir, f"{line_label(line_id)}_daily_unique.csv"),
        #                     index_label="date")

        # --- 2) 각 날짜별 시간대 그래프 저장 ---
        for d in dates:
            day_start = pd.Timestamp(d)
            for f in freqs:
                s = time_bucket_counts(root_path, d, f, line_id)
                ftag = {"h":"1hour","30min":"30min","10min":"10min","1min":"1min"}[f]
                subdir_f = os.path.join(subdir, f)       # plots/lineX/h or 30min or 10min
                os.makedirs(subdir_f, exist_ok=True)
                save_time_series_plot(
                    s,
                    f"{line_label(line_id)} time series ({d}, {ftag})",
                    os.path.join(subdir_f, f"{line_label(line_id)}_{d}_{ftag}.png"),
                    day_start, day_start + pd.Timedelta(days=1)
                )
                # (선택) 데이터도 저장
                s.to_csv(os.path.join(subdir_f, f"{line_label(line_id)}_{d}_{ftag}.csv"),
                         header=["count"])
    print(f"Done: output directory = {out_dir}")

# 실행
# process_all_days()


In [ ]:
import os, sys
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

FONT_PATH = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"

# 1) 폰트 파일을 Matplotlib에 명시적으로 등록
fm.fontManager.addfont(FONT_PATH)

# 2) 폰트 매니저 캐시를 프로세스 내에서 강제 리로드
fm._load_fontmanager(try_read_cache=False)

# 3) 전역 폰트 설정 (가급적 family 이름을 문자열로 직접 지정)
matplotlib.rcParams['font.family'] = 'NanumGothic'
matplotlib.rcParams['font.sans-serif'] = ['NanumGothic']
matplotlib.rcParams['axes.unicode_minus'] = False  # 마이너스 깨짐 방지

print("등록된 나눔 폰트:",
      [f.name for f in fm.fontManager.ttflist if 'Nanum' in f.name])

등록된 나눔 폰트: ['NanumGothic']


In [ ]:
###############
# 특정호선 역별로 처리하는 것
###############

def sanitize_filename(s: str) -> str:
    """역명으로 폴더/파일명 만들 때 안전하게 정제"""
    s = re.sub(r"[\\/:*?\"<>|]", "_", s)
    s = s.replace(" ", "")  # 가독성을 위해 공백 제거(원하면 유지)
    return s

def time_bucket_counts_by_station(root_path: str, yyyymmdd: str, freq: str, line_id: int = 2
                                 ) -> Dict[str, pd.Series]:
    """
    주어진 날짜(yyyymmdd)에 대해 2호선의 역별 시간 버킷 카운트를 반환.
    return: {역명 -> Series(count, index=pd.DatetimeIndex)}
    """
    assert line_id == 2, "이 함수는 2호선 전용입니다."
    day_start = pd.Timestamp(yyyymmdd)
    full_idx = pd.date_range(day_start, day_start + pd.Timedelta(days=1), freq=freq, inclusive="left")

    totals: Dict[str, pd.Series] = {}  # 역명 -> series 누적

    for p in iter_day_files(root_path, yyyymmdd, recursive):
        df = load_part_df(p, day_start)
        if df.empty:
            continue
        df = filter_by_line(df, line_id)
        if df.empty:
            continue

        # InInformation → 4자리 코드 → 역명
        # (성능을 위해 벡터화: apply 한 번으로 code 추출 후 map)
        codes = df["InInformation"].apply(get_line_code_str4)
        names = codes.map(lambda c: code_to_station.get(c, "<UNKNOWN>"))

        tmp = df.assign(station_name=names).dropna(subset=["InTime"])
        if tmp.empty:
            continue

        # 역별로 나눠 resample
        tmp = tmp.set_index("InTime")
        for stn, grp in tmp.groupby("station_name"):
            s = grp.resample(freq).size().astype("int64")
            s = s.reindex(full_idx, fill_value=0).sort_index().astype("int64")
            if stn in totals:
                totals[stn] = totals[stn].add(s, fill_value=0).astype("int64")
            else:
                totals[stn] = s

    # 데이터가 하나도 없었을 경우 대비: 모든 2호선 역에 대해 0 시리즈를 채워줌
    for stn in names_line_2:
        if stn not in totals:
            totals[stn] = pd.Series(0, index=full_idx, dtype="int64")

    return totals


# === 메인 루프 교체: 2호선 각 역별 플롯/CSV 저장 =============================
def process_line2_station_timeseries():
    dates = discover_dates(root_path, recursive)
    if not dates:
        raise RuntimeError("No dates found. Please check file name pattern/path.")

    # 출력 폴더: plots/line2/<freq>/<역명>
    base_subdir = os.path.join(out_dir, "line2")
    os.makedirs(base_subdir, exist_ok=True)

    for d in dates:
        day_start = pd.Timestamp(d)
        for f in freqs:
            totals = time_bucket_counts_by_station(root_path, d, f, line_id=2)

            ftag = {"h":"1hour","30min":"30min","10min":"10min","1min":"1min"}[f]
            subdir_f = os.path.join(base_subdir, f, "subway")
            os.makedirs(subdir_f, exist_ok=True)

            # 역별 저장
            for stn, s in totals.items():
                stn_safe = sanitize_filename(stn)
                stn_dir = os.path.join(subdir_f, stn_safe)
                os.makedirs(stn_dir, exist_ok=True)

                png_path = os.path.join(stn_dir, f"line2_{stn_safe}_{d}_{ftag}.png")
                csv_path = os.path.join(stn_dir, f"line2_{stn_safe}_{d}_{ftag}.csv")

                save_time_series_plot(
                    s,
                    f"2호선 {stn} 역 이용 추이 ({d}, {ftag})",
                    png_path,
                    day_start, day_start + pd.Timedelta(days=1)
                )
                s.to_csv(csv_path, header=["count"])

    print(f"Done: output directory = {out_dir}")

# === 실행 진입점: 기존 process_all_days() 대신 아래 함수 호출 ==================
process_line2_station_timeseries()
# === 추가/교체 코드 끝 =======================================================

Done: output directory = /home/data/202205/train_pars_final_5/plots
